# 4-deg (latitude) x 5-deg (longitude) monthly average gridded datasets for SIF and XCO2 over Continental US

## With $\pm$ {0.5, 1.0} -degree longitude grid shifts included as dataset variables 
---

In [1]:
import sys
sys.path.insert(0, "../src")

import numpy as np
import pandas as pd 
import xarray as xr

from data_utils import regrid

In [2]:
def prep_gridded_df(ds, offset=0):
    lon_lwr = -125+offset 
    lon_upr = -65+offset
    lat_lwr = 12
    lat_upr = 52
    
    df = ds.to_dataframe()
    bounds = (
        (df.lon >= lon_lwr)
        & (df.lon <= lon_upr)
        & (df.lat >= lat_lwr)
        & (df.lat <= lat_upr)
    )
    df = df.loc[bounds].reset_index()

    return (
            regrid(df=df, lon_res=5, lat_res=4, lon_lwr=lon_lwr, lon_upr=lon_upr, lat_lwr=lat_lwr, lat_upr=lat_upr)
            .groupby(["lon", "lat"])
            .resample("1MS", on="time")
            .mean()
            .drop(columns=["lon", "lat"])
            .reset_index()
        )

In [3]:
ds_xco2 = xr.open_dataset("../data/exp_pro/OCO2_Lite_XCO2.nc")
ds_sif = xr.open_dataset("../data/exp_pro/OCO2_Lite_SIF740.nc")

In [4]:
# Compute a gridded dataset for each offset and concatenate
offsets = np.linspace(-1, 1., 5)
df_xco2 = pd.concat([prep_gridded_df(ds_xco2, offset) for offset in offsets])
df_sif = pd.concat([prep_gridded_df(ds_sif, offset) for offset in offsets])

In [5]:
# Merge dataframes
ds_conus_microlag = (
    pd.merge(df_xco2, df_sif, on=["lon", "lat", "time"], how="outer")
    .set_index(["lon", "lat", "time"])
    .to_xarray()
)
ds_conus_microlag

<xarray.Dataset>
Dimensions:   (lat: 10, lon: 60, time: 75)
Coordinates:
  * lon       (lon) float64 -123.5 -123.0 -122.5 -122.0 ... -67.5 -67.0 -66.5
  * lat       (lat) float64 14.0 18.0 22.0 26.0 30.0 34.0 38.0 42.0 46.0 50.0
  * time      (time) datetime64[ns] 2014-09-01 2014-10-01 ... 2020-11-01
Data variables:
    xco2      (lon, lat, time) float32 395.878 nan 396.43573 ... 410.30804 nan
    xco2_var  (lon, lat, time) float32 0.7360611 nan 0.8220609 ... 1.2700796 nan
    sif       (lon, lat, time) float32 nan nan nan ... 0.0632736 0.039882254
    sif_var   (lon, lat, time) float32 nan nan nan ... 0.10670814 0.1726651

In [6]:
ds_conus_microlag.to_netcdf("../data/exp_pro/OCO2_4x5deg_monthly_conus_microlag.nc", format="NETCDF4")
ds_conus_microlag.close()